# Chapter 7 Agent

- [1. Set OpenAI API Key](#1. Set OpenAI-API-Key)
- [2. LangChain built-in tools](#2. LangChain built-in tools)
- [2.1 Use llm-math and wikipedia tools](#2.1-Use llm-math and wikipedia tools)
- [2.2 Use PythonREPLTool tool](#2.2-Use PythonREPLTool tool)
- [3. Define your own tools and use them in agents](#3.-Define your own tools and use them in agents)
- [3.1 Create and use custom time tools](#3.1-Create and use custom time tools)

## 1. Set OpenAI API Key

Log in to [OpenAI account](https://platform.openai.com/account/api-keys) to get API Key, and then set it as an environment variable.

- If you want to set it as a global environment variable, you can refer to [Zhihu article](https://zhuanlan.zhihu.com/p/627665725).

- If you want to set it as a local/project environment variable, create a `.env` file in this file directory, open the file and enter the following content.

<p style="font-family:verdana; font-size:12px;color:green">
OPENAI_API_KEY="your_api_key" 
</p>

Replace "your_api_key" with your own API Key

In [1]:
# Download the required packages python-dotenv and openai
# If you need to view the installation process log, you can remove -q
!pip install -q python-dotenv
!pip install -q openai

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Read local/project environment variables.

# find_dotenv() finds and locates the path of the .env file
# load_dotenv() reads the .env file and loads the environment variables in it into the current running environment
# If you set a global environment variable, this line of code will have no effect.
_ = load_dotenv(find_dotenv())

# Get the environment variable OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']  

## 2. LangChain built-in tools

In [2]:
# If you need to view the installation process log, you can delete -q
# -U installs the latest version of wikipedia. Its function is the same as --upgrade
!pip install -U -q wikipedia
!pip install -q openai

In [3]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

### 2.1 Using llm-math and wikipedia tools

#### 1️⃣ Initialize the large language model

- The default key `openai_api_key` is the environment variable `OPENAI_API_KEY`. So before running the following code, make sure you have set the environment variable `OPENAI_API_KEY`. If you don't have a key yet, please [get your API Key](https://platform.openai.com/account/api-keys) .
- The default model `model_name` is `gpt-3.5-turbo`.
- For more information about the model default parameters, please see [here](https://github.com/hwchase17/langchain/blob/master/langchain/chat_models/openai.py).

In [4]:
# The parameter temperature is set to 0.0 to reduce the randomness of the generated answers.
llm = ChatOpenAI(temperature=0)

#### 2️⃣ Load toolkits
- `llm-math` tool combines language models and calculators for mathematical calculations
- `wikipedia` tool connects to Wikipedia through the API for search queries.

In [5]:
tools = load_tools(
    ["llm-math","wikipedia"], 
    llm=llm #第一步初始化的模型
)

#### 3️⃣ Initialize the agent

- `agent`: Agent type. Here we use `AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION`. Among them, `CHAT` represents that the agent model is optimized for dialogue, and `REACT` represents the prompt template designed for REACT.
- `handle_parsing_errors`: Whether to handle parsing errors. When a parsing error occurs, the error information is returned to the big model for correction.
- `verbose`: Whether to output the intermediate step results.

In [6]:
agent= initialize_agent(
    tools, #第二步加载的工具
    llm, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 4️⃣.1️⃣ Using an agent to answer math questions

In [7]:
agent("计算300的25%，思考过程请使用中文。") 



> Entering new  chain...
Question: 计算300的25%
Thought: 我可以使用计算器来计算这个百分比
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 25 / 100"
}
```

Observation: Answer: 75.0
Thought:我现在知道最终答案了
Final Answer: 300的25%是75.0

> Finished chain.


{'input': '计算300的25%，请用中文', 'output': '300的25%是75.0'}

✅ **Summary**

1. The model gives a thought about what needs to be done next

<p style="font-family:verdana; font-size:12px;color:green"> <strong>Thought</strong>: We need to calculate 25% of 300, and multiplication and division are required in this process. </p>

2. The model takes action based on the thought
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Action</strong>: Use a calculator and enter 300*0.25</p>

3. The model gets an observation
<p style="font-family:verdana; font-size:12px;color:green"><strong>Observation</strong>: Answer: 75.0</p>

4. Based on the observation, the model gives a thought about what needs to be done next
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Thinking</strong>: We have the answer to our question </p>

5. Give the final answer (Final Answer)
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Final answer</strong>: 75.0 </p>
5. Give the final answer in the form of a dictionary.

#### 4️⃣.2️⃣ Tom M. Mitchell’s Books

In [18]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent(question) 



> Entering new  chain...
I need to find out what book Tom M. Mitchell wrote.
Action: Search for Tom M. Mitchell's books.
Action Input: "Tom M. Mitchell books"
Observation: Search for Tom M. Mitchell's books. is not a valid tool, try another one.
Thought:I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

...

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I can try searching for Tom M. Mitchell's books using a search engine.
Action: Search for "Tom M. Mitchell books"
Action Input: "Tom M. Mitchell books"
Observation: Search for "Tom M. Mitchell books" is not a valid tool, try another one.
Thought:

> Finished chain.


{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

✅ **Summary**

1. The model gives a thought about what needs to be done next
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Thought</strong>: I should use Wikipedia to search. </p>

2. The model takes action based on the thought
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Action</strong>: Use Wikipedia and enter Tom M. Mitchell</p>

3. The model gets an observation
<p style="font-family:verdana; font-size:12px;color:green"><strong>Observation</strong>: Page: Tom M. Mitchell, Page: Tom Mitchell (Australian football player)</p>

4. Based on the observation, the model gives a thought about what needs to be done next
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Action</strong>: Use Wikipedia and enter Tom Mitchell</p>:verdana; font-size:12px;color:green"> <strong>Thinking</strong>: Tom M. Mitchell's book is Machine Learning </p>

5. Give the final answer (Final Answer)
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Final Answer</strong>: Machine Learning </p>
5. Give the final answer in the form of a dictionary.

It is worth noting that the process of running the inference of the model each time may be different, but the final result is consistent.

### 2.2 Using PythonREPLTool

#### 1️⃣ Create a Python proxy

In [8]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,  #使用前面一节已经加载的大语言模型
    tool=PythonREPLTool(), #使用Python交互式环境工具（REPLTool）
    verbose=True #输出中间步骤
)

#### 2️⃣ Use a proxy to sort customer names

In [14]:
customer_list = ["小明","小黄","小红","小蓝","小橘","小绿",]

In [22]:
!pip install -q pinyin

In [23]:
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 



> Entering new  chain...
我需要使用拼音库来将客户名字转换为拼音。我可以使用Python的pinyin库来实现这个功能。
Action: Python_REPL
Action Input: import pinyin
Observation: 
Thought:我现在可以使用pinyin库来将客户名字转换为拼音了。
Action: Python_REPL
Action Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']
Observation: 
Thought:我现在可以使用pinyin库将names列表中的客户名字转换为拼音了。
Action: Python_REPL
Action Input: pinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]
Observation: 
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

我已经成功将客户名字转换为拼音并存储在pinyin_names列表中了。
Final Answer: pinyin_names = ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']

> Finished chain.


"pinyin_names = ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaojv', 'xiaolv']"

#### 3️⃣ Use debug mode

Run again in debug mode, we can correspond the above 6 steps to the following specific processes
1. The model gives thoughts on what needs to be done next
- <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor] Entering Chain run with input</p>
- <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input</p>
- <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input</p>
- <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [12.25s] Exiting LLM run with output</p>
- <p style="font-family:verdana; font-size:12px;color:green">[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [12.25s] Exiting Chain run with output</p>
2. The model takes action based on thinking. Because the tools used are different, the output of Action is also different from before. The output here is python code
- <p style="font-family:verdana; font-size:12px;color:green">[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [12.25s] Exiting Chain run with output</p>
2. The model takes action based on thinking. Because the tools used are different, the output of Action is also different from before. The output here is python codelor:green"> [tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL] Entering Tool run with input</p>
- <p style="font-family:verdana; font-size:12px;color:green"> [tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL] [2.2239999999999998ms] Exiting Tool run with output</p>
3. The model gets observations (Observation) 
- <p style="font-family:verdana; font-size:12px;color:green"> [chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain] Entering Chain run with input</p>
4. Based on the observations, the model gives thoughts on what needs to be done next (Thought)- <p style="font-family:verdana; font-size:12px;color:green"> [llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] Entering LLM run with input</p>
- <p style="font-family:verdana; font-size:12px;color:green"> [llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [6.94s] Exiting LLM run with output</p>

5. Give the final answer
- <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [6.94s] Exiting Chain run with output</p>
6. Return the final answer.
- <p style="font-family:verdana; font-size:12px;color:green"> [chain/end] [1:chain:AgentExecutor] [19.20s] Exiting Chain run with output</p>

In [25]:
import langchain
langchain.debug=True
agent.run(f"""在这些客户名字转换为拼音\
并打印输出列表： {customer_list}\
思考过程请使用中文。""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "在这些客户名字转换为拼音并打印输出列表： ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']思考过程请使用中文。",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just retu

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [12.76s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我现在可以使用pinyin库来将客户名字转换为拼音。\nAction: Python_REPL\nAction Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']\npinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]\nprint(pinyin_names)",
        "generation_info": null,
        "message": {
          "content": "我现在可以使用pinyin库来将客户名字转换为拼音。\nAction: Python_REPL\nAction Input: names = ['小明', '小黄', '小红', '小蓝', '小橘', '小绿']\npinyin_names = [pinyin.get(i, format='strip', delimiter='') for i in names]\nprint(pinyin_names)",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 379,
      "completion_tokens": 89,
      "total_tokens": 468
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [12.77s] Exiti

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-2YlJyPMl62f07XPJCAlXfDxj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

[llm/end] [1:chain:AgentExecutor > 8:chain:LLMChain > 9:llm:ChatOpenAI] [20.25s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']",
        "generation_info": null,
        "message": {
          "content": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 500,
      "completion_tokens": 43,
      "total_tokens": 543
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:AgentExecutor > 8:chain:LLMChain] [20.25s] Exiting Chain run with output:
{
  "text": "我现在知道了最终答案。\nFinal Answer: ['xiaoming', 'xiaohuang', 'xiaohong', 'xiaolan', 'xiaoju', 'xiaolv']"
}
[chain/end] [1:chain:AgentExecutor] [35.93s] Exiting Chain run with output:
{
  "o

## 3. Define your own tools and use them in the agent

In [26]:
# If you need to view the installation process log, you can remove -q
!pip install -q DateTime

### 3.1 Creating and using custom time tools

In [27]:
# Import tool function decorator
from langchain.agents import tool
from datetime import date

#### 1️⃣ Use tool function decorator to build custom tools
The tool function decorator can be applied to any function to convert the function into a LongChain tool, making it a tool that can be called by the agent.

We need to add a very detailed documentation string to the function so that the agent knows how to use the function/tool ​​under what circumstances.

For example, the function `time` below, we added a detailed documentation string

```python
"""
Returns today's date, for any problems related to getting today's date.
The input should always be an empty string, and the function will always return today's date.
Any date calculations should be done outside this function.
"""

```

In [28]:
@tool
def time(text: str) -> str:
    """
    返回今天的日期，用于任何需要知道今天日期的问题。\
    输入应该总是一个空字符串，\
    这个函数将总是返回今天的日期，任何日期计算应该在这个函数之外进行。
    """
    return str(date.today())

#### 2️⃣ Initialize the proxy

In [36]:
agent= initialize_agent(
    tools=[time], #将刚刚创建的时间工具加入代理
    llm=llm, #初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

#### 3️⃣ Use the proxy to ask for today's date
**Note**: The proxy may sometimes fail (this feature is under development). If an error occurs, try running it again.

In [37]:
agent("今天的日期是？") 



> Entering new  chain...
根据提供的工具，我们可以使用`time`函数来获取今天的日期。

Thought: 使用`time`函数来获取今天的日期。

Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2023-07-04
Thought:我现在知道了最终答案。
Final Answer: 今天的日期是2023-07-04。

> Finished chain.


{'input': '今天的日期是？', 'output': '今天的日期是2023-07-04。'}

✅ **Summary**

1. The model gives a thought about what needs to be done next
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Thought</strong>: I need to use the time tool to get today's date</p>

2. The model takes action based on the thought. Because the tools used are different, the output of the Action is also different from before. The output here is python code
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Action</strong>: Use the time tool and input an empty string</p>

3. The model gets an observation
<p style="font-family:verdana; font-size:12px;color:green"><strong>Observation</strong>: 2023-07-04</p>

4. Based on the observation, the model gives a thought about what needs to be done next (Thought)
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Thinking</strong>: I have successfully retrieved today's date using the time tool</p>

5. Give the final answer (Final Answer)
<p style="font-family:verdana; font-size:12px;color:green"> <strong>Final Answer</strong>: Today's date is 2023-07-04.</p>
6. Return the final answer.